In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
book_df = pd.read_csv('Books.csv')
ratings_df = pd.read_csv('Ratings.csv').sample(40000)
user_df = pd.read_csv('Users.csv')
user_rating_df = ratings_df.merge(user_df, left_on = 'User-ID', right_on = 'User-ID')

C:\Users\himan\AppData\Local\Temp\ipykernel_29732\1239598326.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  book_df = pd.read_csv('Books.csv')


In [3]:
user_rating_df

,User-ID,ISBN,Book-Rating,Location,Age
0,264152,0393322203,0,"boston, massachusetts, usa",NaN
1,264152,9580600031,10,"boston, massachusetts, usa",NaN
2,264152,0062508687,0,"boston, massachusetts, usa",NaN
3,264152,0896081370,10,"boston, massachusetts, usa",NaN
4,264152,0807047023,0,"boston, massachusetts, usa",NaN
...,...,...,...,...,...
39995,226771,0440983568,0,"pinetop, arizona, usa",45.0
39996,247760,0061052175,0,"marlborough, new hampshire, usa",NaN
39997,231263,0679310711,10,"cornwall, ontario, canada",27.0
39998,37556,0060084456,0,"wilton, maine, usa",NaN


In [4]:
book_user_rating = book_df.merge(user_rating_df, left_on = 'ISBN',right_on = 'ISBN')
book_user_rating = book_user_rating[['ISBN', 'Book-Title', 'Book-Author', 'User-ID', 'Book-Rating']]
book_user_rating.reset_index(drop=True, inplace = True)

In [5]:
d ={}
for i,j in enumerate(book_user_rating.ISBN.unique()):
    d[j] =i
book_user_rating['unique_id_book'] = book_user_rating['ISBN'].map(d)

In [6]:
users_books_pivot_matrix_df = book_user_rating.pivot(index='User-ID', 
                                                          columns='unique_id_book', 
                                                          values='Book-Rating').fillna(0)

In [7]:
users_books_pivot_matrix_df.head()


unique_id_book,0,1,2,3,4,5,6,7,8,9,...,26117,26118,26119,26120,26121,26122,26123,26124,26125,26126
User-ID,,,,,,,,,,,,,,,,,,,,,
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
86,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
99,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
183,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
users_books_pivot_matrix_df = users_books_pivot_matrix_df.values
users_books_pivot_matrix_df

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [9]:
from scipy.sparse.linalg import svds

NUMBER_OF_FACTORS_MF = 15

#Performs matrix factorization of the original user item matrix
U, sigma, Vt = svds(users_books_pivot_matrix_df, k = NUMBER_OF_FACTORS_MF)

In [10]:
sigma = np.diag(sigma)
sigma.shape

(15, 15)

In [11]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) 
all_user_predicted_ratings

array([[-6.76268862e-38, -1.31377625e-35,  1.58818858e-33, ...,
        -7.95370417e-35, -2.80934990e-35,  0.00000000e+00],
       [-4.58725702e-38, -8.74724596e-36,  1.16160730e-33, ...,
        -7.43532879e-35, -1.39170199e-35,  0.00000000e+00],
       [-4.25749627e-37, -3.10444464e-35, -2.02453702e-32, ...,
         2.66185767e-33, -7.37219501e-34,  0.00000000e+00],
       ...,
       [ 5.13089671e-30,  5.28951715e-31,  3.28544283e-25, ...,
        -2.50000316e-26,  1.26465855e-26,  0.00000000e+00],
       [ 1.10632737e-37, -1.00946035e-35,  1.10943237e-33, ...,
        -5.08285351e-35, -2.24961532e-35,  0.00000000e+00],
       [-4.92523372e-37, -2.86927616e-36, -8.14928254e-34, ...,
         7.42786881e-36,  6.82010700e-35,  0.00000000e+00]])

In [12]:
def top_cosine_similarity(data, book_id, top_n=10):
    index = book_id 
    book_row = data[index, :]
    magnitude = np.sqrt(np.einsum('ij, ij -> i', data, data))
    similarity = np.dot(book_row, data.T) / (magnitude[index] * magnitude)
    sort_indexes = np.argsort(-similarity)
    return sort_indexes[:top_n]

def similar_books(book_user_rating, book_id, top_indexes):
    print('Recommendations for {0}: \n'.format(
    book_user_rating[book_user_rating.unique_id_book == book_id]['Book-Title'].values[0]))
    for id in top_indexes + 1:
        print(book_user_rating[book_user_rating.unique_id_book == id]['Book-Title'].values[0])

In [13]:
k = 50
movie_id =25954  
top_n = 3
sliced = Vt.T[:, :k] # representative data

similar_books(book_user_rating, 25954, top_cosine_similarity(sliced, movie_id, top_n))

Recommendations for Chicken Little: 

More Cunning Than Man: A Social History of Rats and Man
Junior Miss
Wanted...: Mud Blossom


C:\Users\himan\AppData\Local\Temp\ipykernel_29732\437162856.py:5: RuntimeWarning: invalid value encountered in divide
  similarity = np.dot(book_row, data.T) / (magnitude[index] * magnitude)
